In [ ]:
import os

directory = "./assets/2002_4"

if not directory:
    directory = input("Input raw dataset directory:")

files = [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

count = 1
if files:
    curr = files[0].split("_")[0]
else:
    print("No files found in the directory.")
    exit()

for file in files:
    file_split = file.split("_")
    if file_split[0] != curr:
        curr = file_split[0]
        count += 1
    
    new_file_name = "_".join(file_split[1:])
    new_file_name = str(count).zfill(3) + "_" + new_file_name
    
    old_path = os.path.join(directory, file)
    new_path = os.path.join(directory, new_file_name)
    
    os.rename(old_path, new_path)
    print(f"Successfully changed file name from ({file}) to ({new_file_name})")

In [29]:
import cv2
import numpy as np
import os
from tqdm import tqdm

def skeletonize(img):
    """Alternative skeletonization implementation without ximgproc"""
    skel = np.zeros(img.shape, np.uint8)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3))
    while True:
        open_img = cv2.morphologyEx(img, cv2.MORPH_OPEN, element)
        temp = cv2.subtract(img, open_img)
        eroded = cv2.erode(img, element)
        skel = cv2.bitwise_or(skel, temp)
        img = eroded.copy()
        if cv2.countNonZero(img) == 0:
            break
    return skel

def process_fingerprint(image_path, output_path, target_size=None, upscale_factor=1.0):
    """
    Enhanced fingerprint processing pipeline with optional upscaling
    """
    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")
    
    # Resize if needed
    if target_size:
        image = cv2.resize(image, target_size, interpolation=cv2.INTER_LANCZOS4)
    elif upscale_factor != 1.0:
        h, w = image.shape
        image = cv2.resize(image, (int(w*upscale_factor), int(h*upscale_factor)), 
                         interpolation=cv2.INTER_LANCZOS4)
    
    # 1. Contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(image)
    
    # 2. Noise reduction
    denoised = cv2.bilateralFilter(enhanced, 9, 75, 75)
    
    # 3. Adaptive thresholding
    thresh = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                cv2.THRESH_BINARY_INV, 21, 7)
    
    # 4. Morphological operations
    kernel = np.ones((3,3), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=1)
    morph = cv2.morphologyEx(morph, cv2.MORPH_OPEN, kernel, iterations=1)
    
    # 5. Skeletonization (using alternative method)
    skeleton = skeletonize(morph)
    
    # 6. Final inversion and saving
    result = cv2.bitwise_not(skeleton)
    cv2.imwrite(output_path, result)

def batch_process_fingerprints(input_folder, output_folder, upscale=False):
    """
    Process all fingerprints in a folder
    """
    os.makedirs(output_folder, exist_ok=True)
    files = [f for f in os.listdir(input_folder) if f.lower().endswith('.tif')]
    
    for filename in tqdm(files, desc="Processing Fingerprints"):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)
        
        try:
            if upscale:
                process_fingerprint(input_path, output_path, upscale_factor=2.0)
            else:
                process_fingerprint(input_path, output_path)
        except Exception as e:
            print(f"\nError processing {filename}: {str(e)}")
            continue

if __name__ == "__main__":
    INPUT_FOLDER = "assets"
    OUTPUT_FOLDER = "processed"
    UPSCALE_IMAGES = True  # Set to True for 2x upscaling
    
    print("Starting fingerprint processing...")
    batch_process_fingerprints(INPUT_FOLDER, OUTPUT_FOLDER, upscale=UPSCALE_IMAGES)
    print("\nProcessing completed successfully!")

Starting fingerprint processing...


Processing Fingerprints: 100%|██████████| 640/640 [00:17<00:00, 35.69it/s]


Processing completed successfully!


In [44]:
import os
import shutil
from tqdm import tqdm

# Define paths
dataset_folder = "processed"
train_folder = "dataset/train"
test_folder = "dataset/test"

# Create train and test folders
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Get all image files
all_files = [f for f in os.listdir(dataset_folder) if f.endswith('.tif')]  # Only get .tif files

# Separate files: _1.tif for test, others for train
test_files = [f for f in all_files if f.endswith("_1.tif")]
train_files = [f for f in all_files]  # Exclude test files from train

print(f"Found {len(all_files)} total fingerprint images")
print(f"- Training images: {len(train_files)}")
print(f"- Test images: {len(test_files)}")

# Function to copy files
def copy_files(file_list, destination_folder):
    for file in tqdm(file_list, desc=f"Copying to {os.path.basename(destination_folder)}"):
        src = os.path.join(dataset_folder, file)
        dest = os.path.join(destination_folder, file)
        shutil.copy2(src, dest)

# Copy files to respective folders
copy_files(train_files, train_folder)
copy_files(test_files, test_folder)

print("\nDataset organization completed:")
print(f"- Training set: {len(train_files)} images (copied to {train_folder})")
print(f"- Test set: {len(test_files)} images (copied to {test_folder})")
print(f"Original files remain intact in {dataset_folder}")

Found 640 total fingerprint images
- Training images: 640
- Test images: 80


Copying to test: 100%|██████████| 80/80 [00:00<00:00, 714.79it/s]


Dataset organization completed:
- Training set: 640 images (copied to dataset/train)
- Test set: 80 images (copied to dataset/test)
Original files remain intact in processed


In [ ]:
import cv2
import numpy as np
import os
import random
from tqdm import tqdm

# Path configuration
test_dir = os.path.join("dataset", "test")

def apply_block_damage(image, block_size=40, num_blocks=10):
    """Apply rectangular block damage to fingerprint image with WHITE blocks"""
    damaged = np.copy(image)
    height, width = image.shape
    
    for _ in range(num_blocks):
        x = random.randint(0, width - block_size)
        y = random.randint(0, height - block_size)
        damaged[y:y+block_size, x:x+block_size] = 255  # White blocks (changed from 0 to 255)
    
    return damaged

# Get all test images
test_images = [f for f in os.listdir(test_dir) if f.endswith("_1.tif")]
print(f"Found {len(test_images)} test images to process")

# Process all images with block damage (overwriting originals)
for filename in tqdm(test_images, desc="Applying block damage"):
    image_path = os.path.join(test_dir, filename)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if image is not None:
        # Apply block damage and overwrite original
        damaged_image = apply_block_damage(image)
        cv2.imwrite(image_path, damaged_image)

print(f"\nSuccessfully applied block damage to {len(test_images)} images")
print(f"Original images in {test_dir} have been overwritten")

Found 80 test images to process


Applying block damage: 100%|██████████| 80/80 [00:01<00:00, 47.21it/s]


Successfully applied block damage to 80 images
Original images in dataset\test have been overwritten
